In [ ]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import random
import os
import glob
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import tqdm.notebook as tq
%matplotlib inline
from mpl_toolkits.axes_grid1 import ImageGrid
import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

from PIL import Image
from IPython.display import display
import cv2
from PIL import ImageFile

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, classification_report, precision_recall_fscore_support, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels
import sys

ImageFile.LOAD_TRUNCATED_IMAGES = True

from torchvision.transforms.functional import InterpolationMode
import pprint

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def path_given_id(id, test=False):
    """
    Returns the full path to the image given the id of the image.
    Parameters:
        - id: The id of the image.
        - test: If True returns the relative path from the test folder. Otherwise, returns the relative path to the image from the training folder.
    Returns:
        - The full relative path to the image with the give in id.
    """
    return IMAGES_PATH + ('train/' if not test else 'test/') + str(id) + '.jpg'

def get_img_array(id, test=False):
    """
    Loads the image from the given id, convert the image to a numpy array and return the numpy array.
    Parameters:
        - id: The id of the image.
        - test: If True, loads the image from the test folder. If False,loads the image from the train folder.
    Returns:
        - The image with the give id as a numpy array.
    """
    img = load_img(path_given_id(id, test), target_size=(224, 224))
    return img_to_array(img)

# preprocess_input(np.expand_dims(get_image_array(id, test), axis=0)) will convert the image into 1,224,224,3 to give to predict.
def process_image(id, test=False):
    return preprocess_input(np.expand_dims(get_img_array(id, test), axis=0))

In [ ]:
IMAGES_PATH = '/content/drive/MyDrive/dog_breed_identification_files/'

labels = pd.read_csv(IMAGES_PATH +'labels.csv')
labelnames = pd.read_csv(IMAGES_PATH  + 'sample_submission.csv').keys()[1:]

In [ ]:
codes = range(len(labelnames))
breed_to_code = dict(zip(labelnames, codes))
code_to_breed = dict(zip(codes, labelnames))

labels['target'] =  [breed_to_code[x] for x in labels.breed]
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

training_data = labels_pivot.sample(frac=0.85)
validation_data = labels_pivot[~labels_pivot['id'].isin(training_data['id'])]
testing_data = training_data.sample(frac=0.25)
training_data = training_data[~training_data['id'].isin(testing_data['id'])]

In [ ]:
img_transform = {
    'valid':transforms.Compose([
        transforms.Resize(size = 224, interpolation=InterpolationMode.BILINEAR),
        transforms.CenterCrop(size = 224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'train':transforms.Compose([
        transforms.RandomResizedCrop(size = 224),
        transforms.RandomRotation(degrees = 30),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  
    ]),
    'test':transforms.Compose([
        transforms.Resize(size = 224, interpolation=InterpolationMode.BILINEAR),
        transforms.CenterCrop(size = 224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
class DogDataset(torch.utils.data.Dataset):
    """
    Create a dataset for pytorch batch loading. This is to load few images into memory at a time instead of all the images at once.
    Extends from torch.utils.data.Dataset
    """
    def __init__(self, images_directory, labels, transform):
        """
        Constructor initialization.
        Params:
            - images_directory: The directory where the images are stored.
            - labels: The image labels
            - transform: The transformations to perform on the data.
        """
        self.images_directory = images_directory
        self.labels = labels
        self.transform = transform


    def __len__(self):
        """
        Returns the total number of samples.
        """
        return len(self.labels)

    
    def __getitem__(self, index):
        if self.labels is not None:
            image_name = f'{self.labels["id"].iloc[index]}.jpg'
            full_image_name = self.images_directory + image_name
            
            final_image = Image.open(full_image_name)
            label = self.labels.iloc[index, 1:].astype('float').to_numpy()
            label = np.argmax(label)
            
            if self.transform:
                final_image = self.transform(final_image)
            
            return [final_image, label]
            

In [ ]:
num_workers = 4
batch_size = 50
use_cuda = torch.cuda.is_available()

train_img = DogDataset(IMAGES_PATH + 'train/', training_data, transform = img_transform['train'])
valid_img = DogDataset(IMAGES_PATH + 'train/', validation_data, transform = img_transform['valid'])
test_img = DogDataset(IMAGES_PATH + 'train/', testing_data, transform = img_transform['test'])


dataloaders={
    'train':torch.utils.data.DataLoader(train_img, batch_size, num_workers = num_workers, shuffle=True),
    'valid':torch.utils.data.DataLoader(valid_img, batch_size, num_workers = num_workers, shuffle=False),
    'test':torch.utils.data.DataLoader(test_img, batch_size, num_workers = num_workers, shuffle=True)
}

In [ ]:
class MyCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(MyCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(96),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 128, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(28 * 28 * 64, 128),
            nn.ReLU()
        )
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128, 128)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
myModel = MyCNN()

if use_cuda:
    myModel = myModel.cuda()

In [ ]:
print(myModel)

MyCNN(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
   

In [ ]:
loss_function = nn.CrossEntropyLoss()

# use Stochastic Gradient Descent to minimize the loss.
optimizer = torch.optim.SGD(myModel.parameters(), lr=0.01, momentum=0.75)

In [ ]:
def train(n_epochs, img_transforms, model, optimizer, criterion, use_cuda):
    """returns trained model"""
    for epoch in range(1, n_epochs+1):
        loss_during_train = 0.0
        loss_during_validation = 0.0
        
        model.train()
        
        for index_batch, (image, label) in enumerate(img_transforms['train']):
            if use_cuda:
                image, label = image.cuda(), label.cuda()
            
            optimizer.zero_grad()
            output = model(image)
            
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            loss_during_train = loss_during_train + ((1 / (index_batch + 1)) * (loss.data - loss_during_train))
            
            if index_batch % 10 == 0:
                print(f'Epoch: {epoch} \tBatch: {index_batch + 1} \tTraining Loss: {loss_during_train:.2f}')
        
        
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in img_transforms['valid']:
                if use_cuda:
                    images = images.cuda()
                    labels = labels.cuda()

                outputs = model(images)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        print(f'Accuracy of the network on the {total} validation images: {100 * correct / total} %') 
        
        
        model.eval()
        for index_batch, (image, label) in enumerate(img_transforms['valid']):
            if use_cuda:
                image, label = image.cuda(), label.cuda()

            output = model(image)
            
            loss = criterion(output, label)
            loss_during_validation = loss_during_validation + ((1 / (index_batch + 1)) * (loss.data - loss_during_validation))
            
        print(f'Epoch: {epoch} \tTraining Loss: {loss_during_train:.2f} \tValidation Loss: {loss_during_validation:.2f}')
        
    return model

In [ ]:
n_epochs = 5

output_model =  train(n_epochs, dataloaders, myModel, optimizer, loss_function, use_cuda)

Epoch: 1 	Batch: 1 	Training Loss: 4.85
Epoch: 1 	Batch: 11 	Training Loss: 4.93
Epoch: 1 	Batch: 21 	Training Loss: 4.87
Epoch: 1 	Batch: 31 	Training Loss: 4.84
Epoch: 1 	Batch: 41 	Training Loss: 4.83
Epoch: 1 	Batch: 51 	Training Loss: 4.82
Epoch: 1 	Batch: 61 	Training Loss: 4.82
Epoch: 1 	Batch: 71 	Training Loss: 4.81
Epoch: 1 	Batch: 81 	Training Loss: 4.81
Epoch: 1 	Batch: 91 	Training Loss: 4.81
Epoch: 1 	Batch: 101 	Training Loss: 4.81
Epoch: 1 	Batch: 111 	Training Loss: 4.81
Epoch: 1 	Batch: 121 	Training Loss: 4.80
Epoch: 1 	Batch: 131 	Training Loss: 4.80
Accuracy of the network on the 1533 validation images: 1.304631441617743 %
Epoch: 1 	Training Loss: 4.80 	Validation Loss: 4.79
Epoch: 2 	Batch: 1 	Training Loss: 4.80
Epoch: 2 	Batch: 11 	Training Loss: 4.79
Epoch: 2 	Batch: 21 	Training Loss: 4.79
Epoch: 2 	Batch: 31 	Training Loss: 4.79
Epoch: 2 	Batch: 41 	Training Loss: 4.79
Epoch: 2 	Batch: 51 	Training Loss: 4.79
Epoch: 2 	Batch: 61 	Training Loss: 4.79
Epoch: 2 

In [ ]:
n_epochs = 10

output_model =  train(n_epochs, dataloaders, myModel, optimizer, loss_function, use_cuda)

Epoch: 1 	Batch: 1 	Training Loss: 4.78
Epoch: 1 	Batch: 11 	Training Loss: 4.79
Epoch: 1 	Batch: 21 	Training Loss: 4.78
Epoch: 1 	Batch: 31 	Training Loss: 4.78
Epoch: 1 	Batch: 41 	Training Loss: 4.78
Epoch: 1 	Batch: 51 	Training Loss: 4.78
Epoch: 1 	Batch: 61 	Training Loss: 4.78
Epoch: 1 	Batch: 71 	Training Loss: 4.78
Epoch: 1 	Batch: 81 	Training Loss: 4.78
Epoch: 1 	Batch: 91 	Training Loss: 4.78
Epoch: 1 	Batch: 101 	Training Loss: 4.78
Epoch: 1 	Batch: 111 	Training Loss: 4.78
Epoch: 1 	Batch: 121 	Training Loss: 4.78
Epoch: 1 	Batch: 131 	Training Loss: 4.78
Accuracy of the network on the 1533 validation images: 0.91324200913242 %
Epoch: 1 	Training Loss: 4.78 	Validation Loss: 4.79
Epoch: 2 	Batch: 1 	Training Loss: 4.75
Epoch: 2 	Batch: 11 	Training Loss: 4.78
Epoch: 2 	Batch: 21 	Training Loss: 4.78
Epoch: 2 	Batch: 31 	Training Loss: 4.78
Epoch: 2 	Batch: 41 	Training Loss: 4.78
Epoch: 2 	Batch: 51 	Training Loss: 4.78
Epoch: 2 	Batch: 61 	Training Loss: 4.78
Epoch: 2 	

# We are getting CUDA error because of lot of parameters. So, let's modify the FC layers sizes so we can build and run the model.

# We can also clearly see that as we decrease the perceptrons we can't see much changes in the accuracy.
# Also, these weights are randomly initialized, so they might take a lot of epochs to improve.
# Finally, we can conclude that using pre trained models is the best possible solution and if we have the memory and computation power we can start our training with the pretrained weights and start training our model from there.